The recipe for today is just doing One-Hot most columns and Thermometer encoding of some ordinal columns. Finally, we apply plain Logistic Regression.

Did anyone have success with more complex methods? Please let us know!

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import scipy
from tqdm import tqdm_notebook as tqdm
from scipy.sparse import *

In [2]:
dd0=pd.read_csv("DataSet/train.csv")
ddtest0=pd.read_csv("DataSet/test.csv")

In [3]:
dd0.target.value_counts()

0    208236
1     91764
Name: target, dtype: int64

In [4]:
dd00=dd0[dd0['target']==0]
dd01=dd0[dd0['target']==1]
dd00=dd00.sample(n=100000,random_state=23)
dd00.shape,dd01.shape

((100000, 25), (91764, 25))

In [8]:
ddall=dd0.append(ddtest0, sort=False)
num_train=len(dd0)
ddall.head()

temp_dd0=dd00.append(dd01,sort=False)
ddall=temp_dd0.append(ddtest0,sort=False)
num_train=len(temp_dd0)
ddall.head(4)

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
291215,291215,0,0,0,F,N,Blue,Trapezoid,Axolotl,China,...,6f16df982,1,Contributor,Cold,e,S,gM,2,3,0.0
46283,46283,0,0,0,F,Y,Blue,Square,Snake,Finland,...,133d2331b,1,Contributor,Hot,c,U,KZ,3,4,0.0
178788,178788,0,0,1,T,Y,Blue,Trapezoid,Lion,Russia,...,394b68eb6,1,Novice,Cold,g,G,cp,2,2,0.0
261462,261462,1,0,0,F,N,Green,Polygon,Lion,Russia,...,7130c09a6,2,Novice,Freezing,f,J,Jc,2,12,0.0


In [9]:
print(temp_dd0.target.value_counts())
# print(ddall.bin_0.value_counts())

0    100000
1     91764
Name: target, dtype: int64


# Crazy feature engineering

In [10]:
# drop_cols=["bin_0"]
drop_cols=[]

# Split 2 Letters; This is the only part which is not generic and would actually require data inspection
ddall["ord_5a"]=ddall["ord_5"].str[0]
ddall["ord_5b"]=ddall["ord_5"].str[1]
drop_cols.append("ord_5")

## Map cat vals which are not in both sets to single values

In [11]:
for col in ["nom_5", "nom_6", "nom_7", "nom_8", "nom_9"]:
    train_vals = set(dd0[col].unique())
    test_vals = set(ddtest0[col].unique())
   
    xor_cat_vals=train_vals ^ test_vals
    if xor_cat_vals:
        ddall.loc[ddall[col].isin(xor_cat_vals), col]="xor"

In [12]:
len(xor_cat_vals)

316

In [13]:
ddall[ddall.nom_9=='xor'].shape

(497, 27)

## One-Hot Encode all

In [14]:
X=ddall[ddall.columns.difference(["id", "target"] + drop_cols)]

In [15]:
X.head(3)

,bin_0,bin_1,bin_2,bin_3,bin_4,day,month,nom_0,nom_1,nom_2,...,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5a,ord_5b
291215,0,0,0,F,N,2,3,Blue,Trapezoid,Axolotl,...,c9d67c6a2,95a0ec044,6f16df982,1,Contributor,Cold,e,S,g,M
46283,0,0,0,F,Y,3,4,Blue,Square,Snake,...,a95d13114,5a3ec6aff,133d2331b,1,Contributor,Hot,c,U,K,Z
178788,0,0,1,T,Y,2,2,Blue,Trapezoid,Lion,...,e14d5f3eb,d28926816,394b68eb6,1,Novice,Cold,g,G,c,p


In [16]:
X_oh=X[X.columns.difference(["ord_1", "ord_4", "ord_5a", "ord_5b", "day", "month"])]
oh1=pd.get_dummies(X_oh, columns=X_oh.columns, drop_first=True, sparse=True)
# ohc1=oh1.sparse.to_coo()
ohc1=oh1.to_coo()

In [17]:
type(ohc1)

scipy.sparse.coo.coo_matrix

## 对于部分有序特征进行编码

In [19]:
from sklearn.base import TransformerMixin
from itertools import repeat
import scipy


class ThermometerEncoder(TransformerMixin):
    """
    Assumes all values are known at fit
    """
    def __init__(self, sort_key=None):
        self.sort_key = sort_key
        self.value_map_ = None
    
    def fit(self, X, y=None):
        self.value_map_ = {val: i for i, val in enumerate(sorted(X.unique(), key=self.sort_key))}
        return self
    
    def transform(self, X, y=None):
        values = X.map(self.value_map_)
        
        possible_values = sorted(self.value_map_.values())
        
        idx1 = []
        idx2 = []
        
        all_indices = np.arange(len(X))
        
        for idx, val in enumerate(possible_values[:-1]):
            new_idxs = all_indices[values > val]
            idx1.extend(new_idxs)
            idx2.extend(repeat(idx, len(new_idxs)))
            
        result = scipy.sparse.coo_matrix(([1] * len(idx1), (idx1, idx2)), shape=(len(X), len(possible_values)), dtype="int8")
            
        return result

In [20]:
thermos=[]
for col in ["ord_1", "ord_2", "ord_3", "ord_4", "ord_5a", "day", "month"]:
    if col=="ord_1":
        sort_key=['Novice', 'Contributor', 'Expert', 'Master', 'Grandmaster'].index
    elif col=="ord_2":
        sort_key=['Freezing', 'Cold', 'Warm', 'Hot', 'Boiling Hot', 'Lava Hot'].index
    elif col in ["ord_3", "ord_4", "ord_5a"]:
        sort_key=str
    elif col in ["day", "month"]:
        sort_key=int
    else:
        raise ValueError(col)
    
    enc=ThermometerEncoder(sort_key=sort_key)
    thermos.append(enc.fit_transform(X[col]))

## Combine sparse matrices

In [21]:
ohc=scipy.sparse.hstack([ohc1] + thermos).tocsr()
display(ohc)

X_train = ohc[:num_train]
X_test = ohc[num_train:]
y_train = dd0["target"].values

<391764x16092 sparse matrix of type '<class 'numpy.int16'>'
	with 27264079 stored elements in Compressed Sparse Row format>

In [22]:
type(X_train)

scipy.sparse.csr.csr_matrix

# Make prediction

In [ ]:
# clf=LogisticRegression(C=0.123456789, solver="lbfgs", max_iter=5000)  # MODEL

# clf.fit(X_train, y_train)

# pred=clf.predict_proba(X_test)[:,1]

# pd.DataFrame({"id": ddtest0["id"], "target": pred}).to_csv("submission.csv", index=False)

In [ ]:
'''0.80815 method'''
# clf1=LogisticRegression(C=0.1, solver="lbfgs", max_iter=5000)  # MODEL
# clf1.fit(X_train, y_train)
# pred1=clf1.predict_proba(X_test)[:,1]

# clf2=LogisticRegression(C=0.1, solver="sag", max_iter=5000)  # MODEL
# clf2.fit(X_train, y_train)
# pred2=clf2.predict_proba(X_test)[:,1]

# clf3=LogisticRegression(C=0.1, solver="saga", max_iter=5000)  # MODEL
# clf3.fit(X_train, y_train)
# pred3=clf3.predict_proba(X_test)[:,1]

In [ ]:
clf1=LogisticRegression(C=0.15, solver="lbfgs", max_iter=1200)  # MODEL
clf1.fit(X_train, y_train)
pred1=clf1.predict_proba(X_test)[:,1]

clf2=LogisticRegression(C=0.15, solver="sag", max_iter=1200)  # MODEL
clf2.fit(X_train, y_train)
pred2=clf2.predict_proba(X_test)[:,1]

clf3=LogisticRegression(C=0.15, solver="saga", max_iter=1200)  # MODEL
clf3.fit(X_train, y_train)
pred3=clf3.predict_proba(X_test)[:,1]

In [ ]:
ans=(pred1+pred2+pred3)/3
pd.DataFrame({"id": ddtest0["id"], "target": ans}).to_csv("submission.csv", index=False)

In [ ]:
len(ans)

# Evaluate

In [ ]:
from sklearn.model_selection import cross_validate

score=cross_validate(clf, X_train, y_train, cv=3, scoring="roc_auc")["test_score"].mean()
print(f"{score:.6f}")